In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset
import torch

C:\Users\danda\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\danda\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
# Load dataset from Hugging Face
dataset = load_dataset("turne292/just_prompt_small")
# Access some of the data
data = dataset["train"]
texts = data['prompt']
max_length = 512
texts = [text[:max_length] for text in texts]
print(texts[77])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/danda/.cache/huggingface/datasets/turne292___csv/turne292--just_prompt_small-797ccd9d697f29f0/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

 r/relationships
TITLE: I [19M] have a problem with being open with my girlfriend [18F] of 7 months about a sexual preference
POST: My girlfriend and I have been together for 7 months now with very few problems, I'd say we have a pretty great relationship, enjoy each other's company and have little to no problem being open about things to each other. Lately one thing has been hindering the last part.

I've had a fetish for several years now (I won't be graphic about that, but I'll go into details if someone


In [4]:
# Perform sentiment analysis
classifier = pipeline("sentiment-analysis", device=0 if device.type == "cuda" else -1)

predictions = classifier(texts)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\danda\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
sample=texts[77]
classifier(sample)

[{'label': 'NEGATIVE', 'score': 0.9258292317390442}]

In [6]:
sample=texts[1034]
classifier(sample)

[{'label': 'NEGATIVE', 'score': 0.7702615857124329}]

In [7]:
# Label the data
labeled_data = []
for text, prediction in zip(texts, predictions):
    if prediction["label"] == "POSITIVE":
        label = "positive"  # positive label
    else:
        label = "negative"  # negative label
    labeled_data.append({"text": text, "label": label})


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split dataset into train and test sets
train_data, test_data = train_test_split(labeled_data, test_size=0.2, random_state=42)

# Convert train and test sets to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Save train and test DataFrames to CSV
train_df.to_csv('train_tldr_sentiment_data_small.csv', index=False)
test_df.to_csv('test_tldr_sentiment_data_small.csv', index=False)